### Paper

*[German also Hallucinates! Inconsistency Detection in News
Summaries with the Absinth Dataset](https://aclanthology.org/2024.lrec-main.680.pdf)*

In [2]:
from transformers import pipeline
from huggingface_hub import InferenceClient
from datasets import load_dataset
from tqdm.auto import tqdm
import random
import pandas as pd

/workspaces/dev/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Model

*[Shahm/bart-german](https://huggingface.co/Shahm/bart-german)*

In [11]:
MAX_LEN = 2048
DATASET_FILENAME = "20min-all-articles.csv"
USED_MODEL = "Shahm/bart-german"
USE_INFERENCE_API = True

In [12]:
# TODO: mistralai/Mixtral-8x22B-Instruct-v0.1 or mistralai/Mixtral-8x7B-Instruct-v0.1
if USE_INFERENCE_API:
    client = InferenceClient(api_key="hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

    messages = [
        { "role": "user", "content": "What is the capital of France?" }
    ]

    stream = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1", 
        messages=messages, 
        max_tokens=500,
        stream=True
    )

    for chunk in stream:
        print(chunk.choices[0].delta.content)
else:
    #summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    #summarizer = pipeline("summarization", model="facebook/mbart-large-cc25")
    summarizer = pipeline("summarization", model=USED_MODEL)

In [13]:
def summarize_text(input_text):
    summary_text = summarizer(input_text[:MAX_LEN], min_length=25, max_length=200)[0]["summary_text"]
    return summary_text

### Dataset

[ZurichNLP/20Minuten](https://github.com/ZurichNLP/20Minuten/tree/main) 
(*[Marie Luise Tono Projektseminar WS2425](https://gitlab.ub.uni-bielefeld.de/projektseminarws24-25/Marie_Luise_Tono_Projektseminar_WS2425/-/blob/main/data/20min-all-articles.csv?ref_type=heads)*)

In [14]:
twenty_minutes_df = pd.read_csv(DATASET_FILENAME)

In [15]:
twenty_minutes_df

,file_name,text,picture_text
0,2020-10-18_6_239897547604.json,Wegen der drastisch steigenden Corona-Zahlen g...,In Paris und acht weiteren französischen Städt...
1,2021-05-11_6_428561171434.json,Die Züge und Busse des Schweizer Regionalverke...,Der RBS ist in mehr als 99 Prozent der Fälle r...
2,2021-12-10_6_678900728110.json,Ein News-Scout aus Blonay VD war um acht Uhr m...,A u t o b a h n a b s c h n i t t z w i s c ...
3,2020-04-15_6_670941183943.json,"Waschmaschinen wissen, was sie tun. 90 Minuten...","Sie waschen, die Maschine meldet eine einzige ..."
4,2020-07-29_6_777680273630.json,Die in Dubai ansässige Airline Emirates will i...,Die «Covid-19-Versicherung» ist vom Antritt de...
...,...,...,...
495,2021-08-24_6_804609690909.json,Die Schweizer Fluggesellschaft Swiss führt ein...,Bei der Swiss herrscht ab 15. November Impfpfl...
496,2021-07-23_6_186662834378.json,Ein Raiffeisen-Banker und seine Ehefrau dürfen...,Eine Frau hat ihrem Banker 60'000 Franken verm...
497,2020-04-14_6_667667443469.json,Der junge Alphirt Erwin Ramsauer verschwand am...,Der junge Alphirt Erwin Ramsauer verschwand am...
498,2020-04-15_6_491496211151.json,Keine Woche ohne Auto-Schlagzeilen mit Justin ...,Und plötzlich stand er still. Da nützt es auch...


In [16]:
def sample_20minutes(n_samples=5):
    dataset_len = len(twenty_minutes_df)
    sample_indices = random.sample(range(dataset_len), n_samples)
    for i in sample_indices:
        cur_text = twenty_minutes_df.loc[i]["text"]
        print(cur_text)
        print()

        summary_text = summarize_text(cur_text)
        print(summary_text)
        print("-"*100)


In [17]:
sample_20minutes(3)

Sie lächelt scheu in die Kamera, wirkt mittelmässig glücklich - vor wenigen Tagen tauchte auf Twitter ein neues Bild der Dubaier Prinzessin Latifa bint Mohammed Al Maktum auf. Das Foto entstand auf Island, Latifa posiert mit ihrem Cousin Marcus Essabri und ihrer Kollegin Sioned Taylor, die auch auf einem Bild im vergangenen Juni in Madrid mit Latifa zu sehen war. Essabri schrieb zum Bild: «Wir hatten ein emotionales Wiedersehen. (...) Es war beruhigend, sie so glücklich, gesund und fokussiert zu sehen.» Der neue, weitere Beweis für das Glück der 35-jährigen Sheikah wurde daraufhin zum Anlass genommen, die Kampagne «Free Latifa» aufzulösen. David Haigh, Mitgründer der Internet-Kampagne, sagte der BBC, die aktuelle Situation der Prinzessin sei «die beste Situation, in der sie sich in Bezug auf Freiheit seit zwei Jahrzehnten befindet». Hat die langjährige Befreiungsgeschichte also ein Happy End? Das ist unklar, denn bis jetzt hat sich Latifa nie selber öffentlich zu Wort gemeldet. Agiert 

### Save Data

In [18]:
summary_df = pd.DataFrame(columns=["id","text","summary","model"])
# (file_name, text, summary, model)
for i in tqdm(range(len(twenty_minutes_df)-450)):
    cur_filename = twenty_minutes_df.loc[i]["file_name"]
    cur_text = twenty_minutes_df.loc[i]["text"]
    cur_summary = summarize_text(cur_text)
    cur_model = USED_MODEL

    summary_df.loc[i] = [i, cur_text, cur_summary, cur_model]

100%|██████████| 50/50 [04:41<00:00,  5.63s/it]


In [9]:
summary_df

,id,original,simplification,summary
0,0,Ich date jetzt schon seit längerer Zeit Timon....,Lars datet seit einiger Zeit Timon. Eigentlich...,"Timon entspricht eigentlich genau dem, was ich..."
1,1,Ich kann mich noch sehr gut an den Moment erin...,"Weil ihr Vater unfruchtbar ist, wurde G.C. 199...",Ich kann mich noch sehr gut an den Moment erin...
2,2,In tausenden Schweizer Privathaushalten kümmer...,"Putzhilfen, Babysitter und sonstige Haushaltsa...",Durchschnittlich bekommen sie für die Arbeit r...
3,3,Im Studio Gewichte stemmen und auf dem Laufban...,Clever-Fit schreibt die Shutdown-Zeit nicht au...,Viele Firmen setzen im Shutdown auf Abo-Verlän...
4,4,Gerade erst wurde Kanye West (43) noch inklusi...,Gerüchten zufolge sollen Kanye West (43) und I...,Nach seiner Scheidung von Reality-Star Kim Kar...
...,...,...,...,...
195,195,"Mit 32 Jahren, nach 18 Jahren als Profi, nach ...",Die Schweizer Tennisspielerin Timea Bacsinszky...,Die Lausannerin Timea Bacsinszky gewinnt die W...
196,196,Laut der grossen Sex-Umfrage von 20 Minuten (s...,Laut der Sex-Umfrage von 20 Minuten hatte fast...,Eine Umfrage von 20 Minuten hatte fast jede*r ...
197,197,Der niederländische Schiedsrichter Björn Kuipe...,Der älteste Schiedsrichter des Turniers pfeift...,Björn Kuipers leitet das Finale der EM an dies...
198,198,Ich habe die Frau online kennengelernt und wir...,Nando (26) lernte sein Date online kennen. Die...,Ich habe die Frau online kennengelernt und wir...


In [10]:
summary_df.to_csv("../data/summary.csv", index=False)